<a href="https://colab.research.google.com/github/Anjali-Kadiyala/cancerClassify/blob/main/cancer_classifier_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this everytime to connect notebook to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd

from glob import glob
from skimage.io import imread
from os import listdir

### No need to run this anymore
This section was used to get the data from kaggle

In [3]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download paultimothymooney/breast-histopathology-images

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists
100% 3.10G/3.10G [00:26<00:00, 121MB/s]
100% 3.10G/3.10G [00:26<00:00, 126MB/s]


In [ ]:
import shutil
dataroot = "/content/drive/Shareddrives/cancerClassify_ML@SJSU/breast-histopathology-images"
shutil.unpack_archive("breast-histopathology-images.zip", dataroot)

# Data exploration

* Data root contains 280 subdirs for 280 patients
* Each subdir contains 2 subsdirs '0' and '1'
* The '0' dir contains images of benign cells
* The '1' dir contains images of cancers cells

In [ ]:
files = listdir(dataroot)
print(len(files))

280


We have 280 patients

In [ ]:
print(len(listdir(dataroot + "10253/0")))

479


In [ ]:
# calculate total images
base_path = dataroot
total_images = 0
for patient in range(len(files)):
    patient_id = files[patient]
    for c in [0,1]:
        if patient_id == "IDC_regular_ps50_idx5":
            continue
        patient_path = base_path + "/" + patient_id
        classification_path = patient_path + "/" + str(c)
        images = listdir(classification_path)
        total_images = total_images + len(images)

In [ ]:
total_images

277524

In [ ]:
data = pd.DataFrame(index=np.arange(0, total_images), columns=["patient_id", "path", "target"])
data

,patient_id,path,target
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
277519,NaN,NaN,NaN
277520,NaN,NaN,NaN
277521,NaN,NaN,NaN
277522,NaN,NaN,NaN


In [ ]:
len(files)

280

In [ ]:
for patient in range(len(files)):
    patient_id = files[patient]
    print(patient_id)
    break

12873


In [ ]:
index = 0
for patient in range(len(files)):
    patient_id = files[patient] ## 10253

    for category in [0,1]:
        if patient_id == "IDC_regular_ps50_idx5":
            continue
        else:
            category_path = base_path + "/" + str(patient_id) + "/" + str(category) + "/"
            images = listdir(category_path)
            for image in range(len(images)):
                image_path = images[image]
                data.iloc[index]["path"] = category_path + image_path
                data.iloc[index]["target"] = category
                data.iloc[index]["patient_id"] = patient_id
                index = index + 1

In [ ]:
data

,patient_id,path,target
0,12873,../content/breast-histopathology-images/12873/...,0
1,12873,../content/breast-histopathology-images/12873/...,0
2,12873,../content/breast-histopathology-images/12873/...,0
3,12873,../content/breast-histopathology-images/12873/...,0
4,12873,../content/breast-histopathology-images/12873/...,0
...,...,...,...
277519,9041,../content/breast-histopathology-images/9041/1...,1
277520,9041,../content/breast-histopathology-images/9041/1...,1
277521,9041,../content/breast-histopathology-images/9041/1...,1
277522,9041,../content/breast-histopathology-images/9041/1...,1


In [ ]:
data.shape

(277524, 3)

## Things to check
* How many +ve vs -ve samples are there
* How many +ve -ve samples per person
* No of images per patients, relate to cancer or not canceros ??
* 

# Objective: Create a model that can classify a single image as cancerous or not cancerous
## False +ve is better than -ve